# Simple Walk 1D

## Create a simple environment where you can walk around

input is the direction up, down, left or right
try to reach a goal
terminate if you reach goal or cross border
reward is -distance for goal and -1 for step

base environment on https://github.com/nicknochnack/ReinforcementLearningCourse/blob/main/Project%203%20-%20Custom%20Environment.ipynb

## 1. Import Dependencies

In [13]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [14]:
class SimpleWalk1D(Env):
    def __init__(self):
        # go left or right
        self.action_space = Discrete(2) # 0: left, 1: right
        
        self.observation_space = MultiDiscrete([10, 10])
        
        self.state = (random.randint(0,9), random.randint(0,9))
        
        # set max steps
        self.steps = 10
        
        
    def step(self, action):
        self.steps -= 1
        
        # calculate correct action
        action2 = action * 2 - 1
        
        # update state
        self.state = (self.state[0] + action2, self.state[1])
        
        if self.state[0] == -1 or self.state[0] == 10:
            # went over bounds
            done = True
            reward = -10
        elif self.state[0] == self.state[1]:
            # reached goal
            done = True
            reward = 100
        elif self.steps == 0:
            # ran out of steps
            done = True
            reward = -10
        else:
            done = False
            reward = -1

        return self.state, reward, done, {}

    def render(self):
        pass
    
    def reset(self):
        self.state = (random.randint(0,9), random.randint(0,9))
        self.steps = 10
        return self.state
        
        
        
        
        
        

In [15]:
env = SimpleWalk1D()
type(env.reset())

tuple

In [16]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-19
Episode:2 Score:-19
Episode:3 Score:100
Episode:4 Score:99
Episode:5 Score:-19


In [17]:
log_path = os.path.join('Training', 'Logs')

In [18]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [24]:
# callback
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

stop_callback = StopTrainingOnRewardThreshold(reward_threshold=97, verbose=1)

eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback,
                             eval_freq=1000,
                             best_model_save_path=os.path.join(log_path, 'best_model'),
                             verbose=1)
                             

In [25]:
model.learn(total_timesteps=40_000, callback=eval_callback)

Logging to Training\Logs\PPO_7
Eval num_timesteps=1000, episode_reward=97.20 +/- 2.64
Episode length: 3.80 +/- 2.64
---------------------------------
| eval/              |          |
|    mean_ep_length  | 3.8      |
|    mean_reward     | 97.2     |
| time/              |          |
|    total_timesteps | 1000     |
---------------------------------
New best mean reward!
Stopping training because the mean reward 97.20  is above the threshold 97


In [22]:
model.save('PPO')

In [23]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(97.0, 2.6076809620810595)